## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-24-11-08-00 +0000,wsj,Auto & Transport Roundup: Market Talk,https://www.wsj.com/business/auto-transport-ro...
1,2026-02-24-11-06-00 +0000,wsj,Home Depot Profit Falls As Home Improvement Do...,https://www.wsj.com/business/retail/home-depot...
2,2026-02-24-11-04-48 +0000,bbc,Who is Peter Mandelson?,https://www.bbc.com/news/articles/c3edlg5w855o...
3,2026-02-24-11-00-20 +0000,wapo,Trump’s newest tariffs could face legal challe...,https://www.washingtonpost.com/business/2026/0...
4,2026-02-24-11-00-00 +0000,nypost,Putin’s freezing campaign backfires spectacula...,https://nypost.com/2026/02/24/world-news/putin...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2533/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,trump,51
177,mexico,25
179,cartel,23
47,new,22
125,union,18


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
116,2026-02-24-02-00-00 +0000,wsj,President Trump will use his State of the Unio...,https://www.wsj.com/politics/policy/trump-to-s...,143
177,2026-02-23-22-42-00 +0000,wsj,The Trump administration is considering new na...,https://www.wsj.com/politics/policy/trump-cons...,133
21,2026-02-24-10-30-00 +0000,wsj,Tuesday’s State of the Union address gives Pre...,https://www.wsj.com/economy/sizing-up-the-u-s-...,126
48,2026-02-24-08-45-39 +0000,wsj,Stock Market Today: Dow Futures Inch Up; Trump...,https://www.wsj.com/livecoverage/stock-market-...,118
164,2026-02-23-23-09-36 +0000,nypost,El Mencho killing marks new war against cartel...,https://nypost.com/2026/02/23/world-news/el-me...,117


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
116,143,2026-02-24-02-00-00 +0000,wsj,President Trump will use his State of the Unio...,https://www.wsj.com/politics/policy/trump-to-s...
347,115,2026-02-23-14-17-01 +0000,nypost,Expert’s chilling warning after killing of car...,https://nypost.com/2026/02/23/world-news/exper...
307,82,2026-02-23-17-28-59 +0000,nypost,Ex-British Ambassador to US Peter Mandelson ar...,https://nypost.com/2026/02/23/world-news/ex-br...
272,66,2026-02-23-19-06-42 +0000,nyt,Top Fed Official Sees Little Effect on Rate Ou...,https://www.nytimes.com/2026/02/23/business/ec...
334,55,2026-02-23-15-34-59 +0000,latimes,Nick Reiner pleads not guilty to murder in kil...,https://www.latimes.com/california/story/2026-...
191,53,2026-02-23-22-17-00 +0000,wsj,Rep. Tony Gonzales of Texas faced calls from R...,https://www.wsj.com/politics/policy/tony-gonza...
48,45,2026-02-24-08-45-39 +0000,wsj,Stock Market Today: Dow Futures Inch Up; Trump...,https://www.wsj.com/livecoverage/stock-market-...
21,39,2026-02-24-10-30-00 +0000,wsj,Tuesday’s State of the Union address gives Pre...,https://www.wsj.com/economy/sizing-up-the-u-s-...
164,36,2026-02-23-23-09-36 +0000,nypost,El Mencho killing marks new war against cartel...,https://nypost.com/2026/02/23/world-news/el-me...
233,35,2026-02-23-20-38-35 +0000,latimes,CBS News cuts ties with longevity expert Peter...,https://www.latimes.com/entertainment-arts/bus...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
